In [ ]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

template = """
I will explain your medical report to you in simple terms.  
- I will speak directly to you as if I am your doctor.  
- All medical terms will be replaced with words that are easy to understand.  
- I will focus on making sure you fully grasp your condition and what it means for your health.  

Conversation History:  
{context}  

Your Report:  
{question}  

Your Health Explained:  


"""

model = OllamaLLM(model="wizardlm2:latest")
prompt = ChatPromptTemplate.from_template(template)

chain = prompt | model



In [ ]:
import easyocr
from transformers import BertTokenizer, BertForMaskedLM
import torch
import pyttsx3
engine = pyttsx3.init()

# Set properties (optional)
engine.setProperty("rate", 150)  # Speed of speech
engine.setProperty("volume", 1.0)  # Volume level


In [ ]:
def extract_text(image_path):
    reader = easyocr.Reader(['en'])  # English language
    results = reader.readtext(image_path, detail=0)  # Extract text
    return " ".join(results)  # Combine extracted text into a string

In [ ]:
image_path = r"C:\Users\hyndh\OneDrive\Desktop\ollama\report.jpg"

extracted_text = extract_text(image_path)
print("Extracted Text:\n", extracted_text)

In [ ]:
def handle_conversation():
    context = ""
    print("Welcome to AI ChatBot!")
    
    user_input = extracted_text  # Assuming extracted_text is defined earlier
    
    result = chain.invoke({"context": context, "question": user_input}).strip()  # Strip unnecessary text
    result = result.split("</think>")[-1].strip() 
    print("Bot:", result)

    context += f"\nUser: {user_input}\nAI: {result}"
    return result,context


In [ ]:
conversation_context, bot_response = handle_conversation()

In [ ]:
import re

def clean_text(text):
    """
    Cleans AI-generated text by removing unwanted symbols like **bold markers**, 
    extra spaces, and unnecessary formatting.
    """
    text = re.sub(r"\*\*(.*?)\*\*", r"\1", text)  # Remove bold markers (**text** -> text)
    text = re.sub(r"_+", " ", text)  # Replace multiple underscores with a space
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces and trim
    return text

In [ ]:
cleaned_text = clean_text(conversation_context)
print(cleaned_text)

In [ ]:
engine.say(cleaned_text)
engine.runAndWait()

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the IndicTrans2 model and tokenizer
model_name = "ai4bharat/indictrans-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def translate_text(text, target_lang="hin"):  # 'hin' for Hindi, 'tam' for Tamil, etc.
    """Translates English medical text to the target Indian language."""
    inputs = tokenizer(f"translate to {target_lang}: {text}", return_tensors="pt")
    outputs = model.generate(**inputs, max_length=1024)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text

# Example: Translate medical report to Hindi
english_text = """Hello there! Let's go through your medical report and explain it in a way that's easy to understand. 
Imagine I'm your doctor, and I'm here to help you understand what your recent hospital stay and procedure involved."""
hindi_translation = translate_text(english_text, "hin")  # 'hin' for Hindi

print("Translated Text:", hindi_translation)


In [ ]:
from mtranslate import translate

def translate_text(text, target_lang="hi"):
    return translate(text, target_lang)

print(translate_text("Hello, how are you?", "hi"))  # Hindi
print(translate_text("Hello, how are you?", "te"))  # Tamil

In [ ]:
translated_text = translate_text(cleaned_text, "te")

In [ ]:
translated_text

In [ ]:
import os
import simpleaudio as sa
from gtts import gTTS
from pydub import AudioSegment

def speak(text, lang="hi"):  # Change "hi" to other Indian languages if needed
    # Convert text to speech
    tts = gTTS(text, lang=lang)
    tts.save("output.mp3")  # Save as MP3

    # Convert MP3 to WAV
    sound = AudioSegment.from_mp3("output.mp3")
    sound.export("output.wav", format="wav")

    # Play WAV file
    wave_obj = sa.WaveObject.from_wave_file("output.wav")
    play_obj = wave_obj.play()
    play_obj.wait_done()

    # Clean up files
    os.remove("output.mp3")
    os.remove("output.wav")

# Speak in Hindi
speak("नमस्ते, आप कैसे हैं?", "hi")


In [ ]:
from vosk import Model, KaldiRecognizer
import wave

model = Model("vosk-model-hi")  # Change to vosk-model-ta, vosk-model-te, etc.
rec = KaldiRecognizer(model, 16000)


In [ ]:
from gtts import gTTS
import os

tts = gTTS(translated_text, lang="te")  # Change "hi" for Hindi, "ta" for Tamil, etc.
tts.save("output.mp3")

# Play the audio
os.system("start output.mp3")  # Windows


In [ ]:
bot_response